In [ ]:
import requests
import bs4
import json

In [ ]:
url = 'http://cyberpunk2021.free.fr/index.php?lng=us'
response = requests.post(url)
bs = bs4.BeautifulSoup(response.text)

In [ ]:
sections = {'weapons':dict()}
# sm13 - ammo, 
# sm1 .. sm8 - weapons

weapon_ids = {'sm1': 'pistols',
 'sm2': 'assault rifles',
 'sm3': 'shotguns',
 'sm4': 'marksman rifles',
 'sm5': 'submachine guns',
 'sm6': 'machine guns',
 'sm7': 'fun guns',
 'sm8': 'weird guns'}
ammo_sect = 'sm13'

for thing in bs.find_all('dd'):
    tp = thing.attrs['id']
    if tp == ammo_sect or tp in weapon_ids:
        cur = {}
        for link in thing.find_all('a'):
            cur[link.text.lower()] = 'http://cyberpunk2021.free.fr/'+link.attrs['href']
    if tp in weapon_ids:
        sections['weapons'][weapon_ids[tp]] = cur
    elif tp == ammo_sect:
        sections['ammo'] = cur

In [ ]:
data = {'visible':True, 'weapons':{'visible':True}}
for weap_type in sections['weapons']:
    data['weapons'][weap_type] = {'visible':True}
    for subsections in sections['weapons'][weap_type]:
        data['weapons'][weap_type][subsections] = {'visible':True}
        
        url = sections['weapons'][weap_type][subsections]
        response = requests.post(url)
        bs = bs4.BeautifulSoup(response.text)
        
        for i in bs.find_all('table', {'class':'e'}):
            cur = {'visible':True, 'itemtype':'TODO'}

            name, img, table, (description) = (list(i.find_all('tr', recursive=False))+[''])[:4]
            cur['name'] = name.text
            
            #cur['img_url'] = 'http://cyberpunk2021.free.fr/'+img.find('img').attrs['src']
            img_url = img.find('img').attrs['src']
            #r = requests.get('http://cyberpunk2021.free.fr/'+img_url)
            #with open(img_url, 'wb') as f:
            #    f.write(r.content)
            cur['img_url'] = img_url
            cur['description'] = description.text if description != '' else ''

            table = table.text.strip()

            lines = table.split('\n')
            for line in lines:
                if not line.isspace() and line != '':
                    param, val = line.split(' : ')
                    cur[param.lower()] = val
            cur['cost'] = float(''.join([i for i in cur['cost'] if (i.isdigit() or i =='-' or i =='.')]))

            data['weapons'][weap_type][subsections][name.text] = cur

In [ ]:
with open('data.json', 'w') as f:
    json.dump(data, f, indent=4)

In [ ]:
url = 'http://cyberpunk2021.free.fr/weapons.php?cat=11&action=show&lng=us'
    
response = requests.post(url)
bs = bs4.BeautifulSoup(response.text)

#r = requests.get('http://cyberpunk2021.free.fr/images/AEK-971.jpg')
# with open('images/AEK-971.jpg', 'wb') as f
#     f.write(r.text)

In [ ]:
for i in bs.find_all('table', {'class':'e'}):
    cur = {'visible':True, 'itemtype':'TODO'}
    
    name, img, table, (description) = (list(i.find_all('tr', recursive=False))+[''])[:4]
    cur['name'] = name.text
    cur['img_url'] = 'http://cyberpunk2021.free.fr/'+img.find('img').attrs['src']
    cur['description'] = description.text if description != '' else ''
    
    table = table.text.strip()
    
    lines = table.split('\n')
    for line in lines:
        if not line.isspace() and line != '':
            param, val = line.split(' : ')
            cur[param.lower()] = val
    
    print(cur)

In [ ]:
def parse_weapons(url):
    res = []
    response = requests.post(url)
    bs = bs4.BeautifulSoup(response.text)

    data = []
    table = bs.find('table')

    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values

    data.pop(0)

    for name, accuracy, avail, conc, mag, ROF, damage, rel, rang, cost in data:
        if 'NA' in cost or 'variable' in cost:
            cost = float('nan')
        else:
            cost = float(''.join([i for i in cost if (i.isdigit() or i =='-' or i =='.')]))

        res.append({'name': name, 'accuracy': accuracy, 'availability':avail, 'concealability': conc, 'magazin size':mag, 'fire rate':ROF, 'damage':damage, 'reliability':rel, 'range':rang, 'cost':cost, 'visible':True})
        
    return res

In [ ]:
#parse ammo table

url = 'http://cyberpunk2021.free.fr/munitions.php?tri=1&stri=asc&lng=us'
response = requests.post(url)
bs = bs4.BeautifulSoup(response.text)

data = []
table = bs.find('table')

rows = table.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

data.pop(0)

ammo = {}

for tipe, name, description, damage, cost in data:
    if tipe not in ammo:
        ammo[tipe] = []
    if 'NA' in cost or 'variable' in cost:
        cost = float('nan')
    else:
        cost = float(''.join([i for i in cost if (i.isdigit() or i =='-' or i =='.')]))
    
    ammo[tipe].append({"name": name, "description":description, "damage":damage, "cost":cost, 'visible':True})
    
with open('ammo_data.json', 'w') as f:
    json.dump(ammo, f)

In [ ]:
def parse_weapons_table(url):
    res = []
    response = requests.post(url)
    bs = bs4.BeautifulSoup(response.text)

    data = []
    table = bs.find('table')

    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values

    data.pop(0)

    for name, accuracy, avail, conc, mag, ROF, damage, rel, rang, cost in data:
        if 'NA' in cost or 'variable' in cost:
            cost = float('nan')
        else:
            cost = float(''.join([i for i in cost if (i.isdigit() or i =='-' or i =='.')]))

        res.append({'name': name, 'accuracy': accuracy, 'availability':avail, 'concealability': conc, 'magazin size':mag, 'fire rate':ROF, 'damage':damage, 'reliability':rel, 'range':rang, 'cost':cost, 'visible':True})
        
    return res

In [ ]:
weapons = {}
urls = {'pistols': 'http://cyberpunk2021.free.fr/tables.php?cat=1&gcat=1&lng=us',
        'assault rifles': 'http://cyberpunk2021.free.fr/tables.php?cat=10&gcat=2&lng=us',
        'shotguns': 'http://cyberpunk2021.free.fr/tables.php?cat=15&gcat=3&lng=us',
        'other rifles': 'http://cyberpunk2021.free.fr/tables.php?cat=19&gcat=4&lng=us',
        'submachine gun': 'http://cyberpunk2021.free.fr/tables.php?cat=24&gcat=5&lng=us',
        'machineguns': 'http://cyberpunk2021.free.fr/tables.php?cat=30&gcat=6&lng=us',
        'heavy weapons': 'http://cyberpunk2021.free.fr/tables.php?cat=35&gcat=7&lng=us',
        'exotic weapons': 'http://cyberpunk2021.free.fr/tables.php?cat=46&gcat=8&lng=us'}

for tp in urls:
    weapons[tp] = parse_weapons(urls[tp])

In [ ]:
with open('weapons.json', 'w') as f:
    json.dump(weapons, f)